In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [29]:
import os

In [156]:
# REGI 데이터 가져오기
datas = os.listdir('data/REGI')
li = []
for d in datas:
    df = pd.read_csv(f"data/REGI/{d}")
    li.append(df)

In [157]:
# 합병, 저장
regi = pd.concat(li)
regi.to_csv('./prepro_data/REGI_data.csv')

In [243]:
import requests
from bs4 import BeautifulSoup
from html_table_parser import parser_functions

ImportError: cannot import name 'Stylesheet' from 'bs4.element' (C:\Users\bitcamp\anaconda3\lib\site-packages\bs4\element.py)

In [34]:
# 재무제표 데이터 크롤링
url = 'https://www.choicestock.co.kr/search/invest/TSLA/MRQ'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
datas = soup.select('table.tableRanking')

In [ ]:
# 데이터 프레임 생성
li = []
for data in datas:
    table = parser_functions.make2d(data)
    df = pd.DataFrame(data=table[1:], columns=table[0])
    li.append(df)

In [ ]:
# 합병
df = pd.concat(li)

In [ ]:
# 저장
df.to_csv('prepro_data/fn_statement.csv')

In [163]:
# Date 전처리 함수
def change_date(x):
    ret = []
    for da in x.split():
        ret.append(da[:-1])
    return '.'.join(ret)

In [183]:
# Volumn 전치리 함수
def m_to_k(x):
    if x[-1] == 'M':
        return float(x[:-1]) * 1000
    elif x[-1] == 'B':
        return float(x[:-1]) * 1000000
    elif x[-1] == 'K':
        return float(x[:-1])
    else:
        return np.nan

In [244]:
# 알루미늄 전처리
df1 = pd.read_csv('data/알루미늄 내역.csv')
df1.drop(['오픈', '고가', '저가'], axis=1, inplace=True)
df1.columns = ['date', 'alu_price', 'alu_vol', 'alu_change']

In [245]:
df1.date = df1.date.apply(change_date)

In [246]:
df1.date = pd.to_datetime(df1.date)

In [247]:
df1['alu_price'] = df1['alu_price'].apply(lambda x: x.replace(',', ''))

In [248]:
df1['alu_price'] = pd.to_numeric(df1['alu_price'])

In [249]:
df1['alu_vol'] = df1['alu_vol'].apply(m_to_k)

In [250]:
df1['alu_change'] = df1['alu_change'].apply(lambda x: float(x.replace('%', '')))

In [251]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1815 entries, 0 to 1814
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        1815 non-null   datetime64[ns]
 1   alu_price   1815 non-null   float64       
 2   alu_vol     1814 non-null   float64       
 3   alu_change  1815 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 56.8 KB


In [252]:
# 구리 데이터 전처리
df2 = pd.read_csv('data/영국 구리 선물 내역.csv')
df2.drop(['오픈', '고가', '저가', '거래량'], axis=1, inplace=True)
df2.columns = ['date', 'cu_price', 'cu_change']

In [256]:
df2.date = df2.date.apply(change_date)

In [257]:
df2.date = pd.to_datetime(df2.date)

In [258]:
df2.cu_price = df2.cu_price.apply(lambda x: float(x.replace(',', '')))

In [259]:
df2['cu_change'] = df2['cu_change'].apply(lambda x: float(x.replace('%', '')))

In [181]:
# 알루미늄, 구리 데이터 합병
df_mg = pd.merge(df1, df2, on='date', how='outer')

In [194]:
# REGI 데이터 전처리
df3 = pd.read_csv('prepro_data/REGI_data.csv')
df3.drop(['Unnamed: 0', '오픈', '고가', '저가'], axis=1, inplace=True)
df3.columns = ['date', 'REGI_price', 'REGI_vol', 'REGI_change']

In [200]:
df3.date = df3.date.apply(change_date)

In [201]:
df3.date = pd.to_datetime(df3.date)

In [204]:
df3.REGI_vol = df3.REGI_vol.apply(m_to_k)

In [205]:
df3['REGI_change'] = df3['REGI_change'].apply(lambda x: float(x.replace('%', '')))

In [208]:
# REGI 합병
df_mg1 = pd.merge(df_mg, df3, on='date', how='outer')

In [211]:
# 합병한 데이터프레임 저장
df_mg1.to_csv('./prepro_data/alu_cu_REGI_df.csv')

In [ ]:
# 리튬 전처리 및 저장

In [222]:
df = pd.read_excel('data/리튬+가격2022-03-08.xls')

In [224]:
df.drop(['등락가', '등락비율', '재고량'], axis=1, inplace=True)

In [226]:
df.columns = ['date', 'li_price']

In [231]:
df.date = pd.to_datetime(df.date)

In [234]:
df.to_csv('prepro_data/data_li.csv')